In [7]:
import numpy as np
import pandas as pd
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import nltk
import datetime as dt
nltk.download("movie_reviews")
nltk.download("punkt")
import tqdm
import matplotlib as plt

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
movie = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv")

In [9]:
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [10]:
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
print(movie.isnull().sum())
print('Duplicados')
print(movie.duplicated().sum())

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64
Duplicados
0


In [12]:
print(credit.isnull().sum())
print('Duplicados')
credit.duplicated().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64
Duplicados


0

In [13]:
df_movies = movie.merge(credit, left_on='id', right_on= 'movie_id')

In [14]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [15]:
df_movies.drop(['title_y', 'movie_id', 'spoken_languages', 'status', 'title_x', 'homepage'], axis = 1, inplace = True)


In [16]:
df_movies.columns


Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'tagline',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [17]:
df_movies.columns = ['presupuesto', 'generos','id', 'keywords', 'idioma', 'titulo', 'sinopsis', 'popularidad', 'productora', 'pais', 'fecha', 'recaudacion', 'duracion', 'tagline', 'rate', 'rate_count', 'cast', 'director' ]

In [18]:
df_movies['sinopsis'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [19]:
df_movies.iloc[0,1]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [20]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [21]:
def clean(text):
    x = []
    for i in ast.literal_eval(text):
        x.append(i['name'])
    y=x[:]
    return y


In [22]:
df_movies['generos']=df_movies['generos'].apply(clean)
df_movies['productora']=df_movies['productora'].apply(clean)
df_movies['pais']=df_movies['pais'].apply(clean)
df_movies

,presupuesto,generos,id,keywords,idioma,titulo,sinopsis,popularidad,productora,pais,fecha,recaudacion,duracion,tagline,rate,rate_count,cast,director
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[Action, Crime, Thriller]",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,[Columbia Pictures],"[Mexico, United States of America]",1992-09-04,2040920,81.0,"He didn't come looking for trouble, but troubl...",6.6,238,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,9000,"[Comedy, Romance]",72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,A newlywed couple's honeymoon is upended by th...,5.9,5,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,0,"[Comedy, Drama, Romance, TV Movie]",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[Front Street Pictures, Muse Entertainment Ent...",[United States of America],2013-10-13,0,120.0,NaN,7.0,6,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,0,[],126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[United States of America, China]",2012-05-03,0,98.0,A New Yorker in Shanghai,5.7,7,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [23]:
def reducir_cast(text):
    x=[]
    counter=0
    for i in ast.literal_eval(text):
        if counter == 3:
            break
        x.append(i['name'])
        counter += 1
            
    y=x[:]
    return y

In [24]:
df_movies['cast']=df_movies['cast'].apply(reducir_cast)
df_movies

,presupuesto,generos,id,keywords,idioma,titulo,sinopsis,popularidad,productora,pais,fecha,recaudacion,duracion,tagline,rate,rate_count,cast,director
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[Action, Crime, Thriller]",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,[Columbia Pictures],"[Mexico, United States of America]",1992-09-04,2040920,81.0,"He didn't come looking for trouble, but troubl...",6.6,238,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,9000,"[Comedy, Romance]",72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,A newlywed couple's honeymoon is upended by th...,5.9,5,"[Edward Burns, Kerry Bishé, Marsha Dietlein]","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,0,"[Comedy, Drama, Romance, TV Movie]",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[Front Street Pictures, Muse Entertainment Ent...",[United States of America],2013-10-13,0,120.0,NaN,7.0,6,"[Eric Mabius, Kristin Booth, Crystal Lowe]","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,0,[],126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[United States of America, China]",2012-05-03,0,98.0,A New Yorker in Shanghai,5.7,7,"[Daniel Henney, Eliza Coupe, Bill Paxton]","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [25]:
def buscar_director(text):
    x=[]
    for i in ast.literal_eval(text):
        if i['job']=="Director":
            x.append(i['name'])
    y=x[:]
    return y

In [26]:
df_movies['director']=df_movies['director'].apply(buscar_director)

In [27]:
df_movies

,presupuesto,generos,id,keywords,idioma,titulo,sinopsis,popularidad,productora,pais,fecha,recaudacion,duracion,tagline,rate,rate_count,cast,director
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,300000000,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,245000000,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[Action, Crime, Thriller]",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,[Columbia Pictures],"[Mexico, United States of America]",1992-09-04,2040920,81.0,"He didn't come looking for trouble, but troubl...",6.6,238,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4799,9000,"[Comedy, Romance]",72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,A newlywed couple's honeymoon is upended by th...,5.9,5,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",[Edward Burns]
4800,0,"[Comedy, Drama, Romance, TV Movie]",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[Front Street Pictures, Muse Entertainment Ent...",[United States of America],2013-10-13,0,120.0,NaN,7.0,6,"[Eric Mabius, Kristin Booth, Crystal Lowe]",[Scott Smith]
4801,0,[],126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[United States of America, China]",2012-05-03,0,98.0,A New Yorker in Shanghai,5.7,7,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[Daniel Hsia]


In [28]:
df_movies.dropna(subset = ['sinopsis'],inplace=True)

In [29]:
df_movies['sinopsis']= df_movies['sinopsis'].apply(lambda x: x.split())

In [30]:
df_movies['sinopsis']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4798    [El, Mariachi, just, wants, to, play, his, gui...
4799    [A, newlywed, couple's, honeymoon, is, upended...
4800    ["Signed,, Sealed,, Delivered", introduces, a,...
4801    [When, ambitious, New, York, attorney, Sam, is...
4802    [Ever, since, the, second, grade, when, he, fi...
Name: sinopsis, Length: 4800, dtype: object

In [31]:
def keyword(text):
    x = []
    for i in ast.literal_eval(text):
        x.append(i['name'])
    y=x[:]
    return y

In [32]:
df_movies['keywords']= df_movies['keywords'].apply(keyword)


In [33]:
df_movies

,presupuesto,generos,id,keywords,idioma,titulo,sinopsis,popularidad,productora,pais,fecha,recaudacion,duracion,tagline,rate,rate_count,cast,director
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,"[A, cryptic, message, from, Bond’s, past, send...",107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"[John, Carter, is, a, war-weary,, former, mili...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",es,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...",14.269792,[Columbia Pictures],"[Mexico, United States of America]",1992-09-04,2040920,81.0,"He didn't come looking for trouble, but troubl...",6.6,238,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4799,9000,"[Comedy, Romance]",72766,[],en,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...",0.642552,[],[],2011-12-26,0,85.0,A newlywed couple's honeymoon is upended by th...,5.9,5,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",[Edward Burns]
4800,0,"[Comedy, Drama, Romance, TV Movie]",231617,"[date, love at first sight, narration, investi...",en,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,...",1.444476,"[Front Street Pictures, Muse Entertainment Ent...",[United States of America],2013-10-13,0,120.0,NaN,7.0,6,"[Eric Mabius, Kristin Booth, Crystal Lowe]",[Scott Smith]
4801,0,[],126186,[],en,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is...",0.857008,[],"[United States of America, China]",2012-05-03,0,98.0,A New Yorker in Shanghai,5.7,7,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[Daniel Hsia]


In [34]:
def limpiar_espacios(lista):
    x=[]
    for i in lista:
        x.append(i.replace(" ",""))
    y = x[:]
    return y


In [35]:
df_movies['generos']= df_movies['generos'].apply(limpiar_espacios)
df_movies['cast']= df_movies['cast'].apply(limpiar_espacios)
df_movies['director']= df_movies['director'].apply(limpiar_espacios)
df_movies['sinopsis']= df_movies['sinopsis'].apply(limpiar_espacios)


In [36]:
df_movies['generos']

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
4798                       [Action, Crime, Thriller]
4799                               [Comedy, Romance]
4800               [Comedy, Drama, Romance, TVMovie]
4801                                              []
4802                                   [Documentary]
Name: generos, Length: 4800, dtype: object

In [37]:
df_movies

,presupuesto,generos,id,keywords,idioma,titulo,sinopsis,popularidad,productora,pais,fecha,recaudacion,duracion,tagline,rate,rate_count,cast,director
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,Enter the World of Pandora.,7.2,11800,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,"[A, cryptic, message, from, Bond’s, past, send...",107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,A Plan No One Escapes,6.3,4466,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,The Legend Ends,7.6,9106,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,260000000,"[Action, Adventure, ScienceFiction]",49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"[John, Carter, is, a, war-weary,, former, mili...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,"Lost in our world, found in another.",6.1,2124,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[Action, Crime, Thriller]",9367,"[united states–mexico barrier, legs, arms, pap...",es,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...",14.269792,[Columbia Pictures],"[Mexico, United States of America]",1992-09-04,2040920,81.0,"He didn't come looking for trouble, but troubl...",6.6,238,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt]",[RobertRodriguez]
4799,9000,"[Comedy, Romance]",72766,[],en,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...",0.642552,[],[],2011-12-26,0,85.0,A newlywed couple's honeymoon is upended by th...,5.9,5,"[EdwardBurns, KerryBishé, MarshaDietlein]",[EdwardBurns]
4800,0,"[Comedy, Drama, Romance, TVMovie]",231617,"[date, love at first sight, narration, investi...",en,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,...",1.444476,"[Front Street Pictures, Muse Entertainment Ent...",[United States of America],2013-10-13,0,120.0,NaN,7.0,6,"[EricMabius, KristinBooth, CrystalLowe]",[ScottSmith]
4801,0,[],126186,[],en,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is...",0.857008,[],"[United States of America, China]",2012-05-03,0,98.0,A New Yorker in Shanghai,5.7,7,"[DanielHenney, ElizaCoupe, BillPaxton]",[DanielHsia]


In [38]:
df_movies['datos'] = df_movies['generos']+df_movies['cast']+df_movies['director']+df_movies['sinopsis']

In [39]:
df_movies['datos']

0       [Action, Adventure, Fantasy, ScienceFiction, S...
1       [Adventure, Fantasy, Action, JohnnyDepp, Orlan...
2       [Action, Adventure, Crime, DanielCraig, Christ...
3       [Action, Crime, Drama, Thriller, ChristianBale...
4       [Action, Adventure, ScienceFiction, TaylorKits...
                              ...                        
4798    [Action, Crime, Thriller, CarlosGallardo, Jaim...
4799    [Comedy, Romance, EdwardBurns, KerryBishé, Mar...
4800    [Comedy, Drama, Romance, TVMovie, EricMabius, ...
4801    [DanielHenney, ElizaCoupe, BillPaxton, DanielH...
4802    [Documentary, DrewBarrymore, BrianHerzlinger, ...
Name: datos, Length: 4800, dtype: object

In [40]:
df_palabras= df_movies[['titulo','datos','fecha','duracion','rate','rate_count','sinopsis','generos']]
df_palabras['fecha'] = pd.to_datetime(df_palabras['fecha'])


<ipython-input-40-1afc5a921506>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['fecha'] = pd.to_datetime(df_palabras['fecha'])


In [41]:
df_palabras['fecha'] = df_palabras['fecha'].dt.year

<ipython-input-41-2e79795a43f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['fecha'] = df_palabras['fecha'].dt.year


In [42]:
df_palabras.head()

,titulo,datos,fecha,duracion,rate,rate_count,sinopsis,generos
0,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, S...",2009.0,162.0,7.2,11800,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]"
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, JohnnyDepp, Orlan...",2007.0,169.0,6.9,4500,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]"
2,Spectre,"[Action, Adventure, Crime, DanielCraig, Christ...",2015.0,148.0,6.3,4466,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, ChristianBale...",2012.0,165.0,7.6,9106,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]"
4,John Carter,"[Action, Adventure, ScienceFiction, TaylorKits...",2012.0,132.0,6.1,2124,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]"


In [43]:
def time(data):
    if data>=150 :
        return("Más de 2h30")
    elif(data>=120) and (data< 150):
        return("Entre 2h y 2h30")
    elif(data>=90) and (data< 120):
        return("Entre 1h30 y 2h")
    else:
        return("Menos de 1h30")


df_palabras['duracion'] = df_palabras['duracion'] .apply(time)


<ipython-input-43-a7ce4af22bf0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['duracion'] = df_palabras['duracion'] .apply(time)


In [44]:
import collections

In [45]:
tiempocount = collections.Counter(df_palabras['duracion'])
tiempocount

Counter({'Más de 2h30': 179,
         'Entre 2h y 2h30': 894,
         'Entre 1h30 y 2h': 3023,
         'Menos de 1h30': 704})

In [46]:
def rate(data):
    if data>=7 :
        return("Buena")
    else:
        return("Mala")


df_palabras['rate'] = df_palabras['rate'] .apply(rate)


<ipython-input-46-e3e753eeb201>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['rate'] = df_palabras['rate'] .apply(rate)


In [47]:
ratecount = collections.Counter(df_palabras['rate'])
ratecount

Counter({'Buena': 986, 'Mala': 3814})

In [48]:
df_palabras['rate_count'].describe()

count     4800.000000
mean       690.645208
std       1234.853376
min          0.000000
25%         54.000000
50%        236.000000
75%        737.250000
max      13752.000000
Name: rate_count, dtype: float64

In [49]:
def pop(data):
    if data>=733 :
        return("Popular")
    else:
        return("No popular")


df_palabras['rate_count'] = df_palabras['rate_count'] .apply(pop)


<ipython-input-49-0479940fce86>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['rate_count'] = df_palabras['rate_count'] .apply(pop)


In [50]:
popcount = collections.Counter(df_palabras['rate_count'])
popcount

Counter({'Popular': 1209, 'No popular': 3591})

In [51]:
df_palabras

,titulo,datos,fecha,duracion,rate,rate_count,sinopsis,generos
0,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, S...",2009.0,Más de 2h30,Buena,Popular,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]"
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, JohnnyDepp, Orlan...",2007.0,Más de 2h30,Mala,Popular,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]"
2,Spectre,"[Action, Adventure, Crime, DanielCraig, Christ...",2015.0,Entre 2h y 2h30,Mala,Popular,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, ChristianBale...",2012.0,Más de 2h30,Buena,Popular,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]"
4,John Carter,"[Action, Adventure, ScienceFiction, TaylorKits...",2012.0,Entre 2h y 2h30,Mala,Popular,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]"
...,...,...,...,...,...,...,...,...
4798,El Mariachi,"[Action, Crime, Thriller, CarlosGallardo, Jaim...",1992.0,Menos de 1h30,Mala,No popular,"[El, Mariachi, just, wants, to, play, his, gui...","[Action, Crime, Thriller]"
4799,Newlyweds,"[Comedy, Romance, EdwardBurns, KerryBishé, Mar...",2011.0,Menos de 1h30,Mala,No popular,"[A, newlywed, couple's, honeymoon, is, upended...","[Comedy, Romance]"
4800,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TVMovie, EricMabius, ...",2013.0,Entre 2h y 2h30,Buena,No popular,"[""Signed,, Sealed,, Delivered"", introduces, a,...","[Comedy, Drama, Romance, TVMovie]"
4801,Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton, DanielH...",2012.0,Entre 1h30 y 2h,Mala,No popular,"[When, ambitious, New, York, attorney, Sam, is...",[]


In [52]:
print(df_palabras['datos'].head())

0    [Action, Adventure, Fantasy, ScienceFiction, S...
1    [Adventure, Fantasy, Action, JohnnyDepp, Orlan...
2    [Action, Adventure, Crime, DanielCraig, Christ...
3    [Action, Crime, Drama, Thriller, ChristianBale...
4    [Action, Adventure, ScienceFiction, TaylorKits...
Name: datos, dtype: object


In [53]:
def sacarcorchetes(lista):
    str=" "
    return (str.join(lista))
    

In [54]:
df_palabras['datos'] = df_palabras['datos'].apply(sacarcorchetes)
df_palabras['datos'] = df_palabras['datos'].apply(lambda x:x.lower())
df_palabras['sinopsis'] = df_palabras['sinopsis'].apply(sacarcorchetes)
df_palabras['sinopsis'] = df_palabras['sinopsis'].apply(lambda x:x.lower())
df_palabras['generos'] = df_palabras['generos'].apply(sacarcorchetes)
df_palabras['generos'] = df_palabras['generos'].apply(lambda x: x.lower())

<ipython-input-54-6742cf72185e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['datos'] = df_palabras['datos'].apply(sacarcorchetes)
<ipython-input-54-6742cf72185e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palabras['datos'] = df_palabras['datos'].apply(lambda x:x.lower())
<ipython-input-54-6742cf72185e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [55]:
lista_generos = df_palabras['generos'].unique().tolist()

lista_generos

['action adventure fantasy sciencefiction',
 'adventure fantasy action',
 'action adventure crime',
 'action crime drama thriller',
 'action adventure sciencefiction',
 'fantasy action adventure',
 'animation family',
 'adventure fantasy family',
 'action adventure fantasy',
 'adventure fantasy action sciencefiction',
 'adventure action thriller crime',
 'action adventure western',
 'adventure family fantasy',
 'sciencefiction action adventure',
 'adventure action fantasy',
 'action comedy sciencefiction',
 'action adventure',
 'adventure fantasy',
 'adventure drama action',
 'drama romance thriller',
 'adventure action sciencefiction',
 'thriller action adventure sciencefiction',
 'action adventure sciencefiction thriller',
 'action adventure thriller',
 'family fantasy adventure',
 'adventure action sciencefiction thriller',
 'fantasy adventure family',
 'animation family adventure comedy',
 'adventure action thriller sciencefiction',
 'animation family comedy',
 'action scienceficti

In [56]:
df_palabras.head()

,titulo,datos,fecha,duracion,rate,rate_count,sinopsis,generos
0,Avatar,action adventure fantasy sciencefiction samwor...,2009.0,Más de 2h30,Buena,Popular,"in the 22nd century, a paraplegic marine is di...",action adventure fantasy sciencefiction
1,Pirates of the Caribbean: At World's End,adventure fantasy action johnnydepp orlandoblo...,2007.0,Más de 2h30,Mala,Popular,"captain barbossa, long believed to be dead, ha...",adventure fantasy action
2,Spectre,action adventure crime danielcraig christophwa...,2015.0,Entre 2h y 2h30,Mala,Popular,a cryptic message from bond’s past sends him o...,action adventure crime
3,The Dark Knight Rises,action crime drama thriller christianbale mich...,2012.0,Más de 2h30,Buena,Popular,following the death of district attorney harve...,action crime drama thriller
4,John Carter,action adventure sciencefiction taylorkitsch l...,2012.0,Entre 2h y 2h30,Mala,Popular,"john carter is a war-weary, former military ca...",action adventure sciencefiction


In [57]:
with open('pelis.pk', 'wb') as pelis:
    pickle.dump(df_palabras, pelis)

NameError: name 'pickle' is not defined

In [58]:
from tqdm import tqdm
from joblib import Parallel, delayed
import pickle

In [59]:
sinopsis_sent

NameError: name 'sinopsis_sent' is not defined

In [60]:
sinopsis_sent['sinopsis'].isnull().sum()

NameError: name 'sinopsis_sent' is not defined

In [ ]:

print("Emocionantes", sinopsis_sent[sinopsis_sent['sentimiento']==0].shape[0])
print("Divertidas", sinopsis_sent[sinopsis_sent['sentimiento']==1].shape[0])


In [ ]:
word_counts = sinopsis_sent['sinopsis'].apply(lambda x: len(x.split()))

word_counts.describe()

In [ ]:
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words('english') 

lemmatizer = WordNetLemmatizer()

def data_preprocessing(sinopsis):
    
    # data cleaning
    sinopsis = re.sub(re.compile('<.*?>'), '', sinopsis) 
    review =  re.sub('[^A-Za-z0-9]+', ' ', sinopsis) 
  
    # lowercase
    sinopsis = sinopsis.lower()
  
    # tokenization
    tokens = nltk.word_tokenize(sinopsis) 
  
    # stop_words removal
    sinopsis = [word for word in tokens if word not in stop_words] s
  
    # lemmatization
    sinopsis = [lemmatizer.lemmatize(word) for word in sinopsis]
  
    # join words in preprocessed review
    sinopsis = ' '.join(sinopsis)
  
    return sinopsis

In [ ]:
sinopsis_sent['sinopsis_ok'] = sinopsis_sent['sinopsis'].apply(lambda sinopsis: data_preprocessing(sinopsis))
sinopsis_sent.head()


In [ ]:
from sklearn.model_selection import train_test_split

data = sinopsis_sent.copy()
y = data['sentimiento'].values
data.drop(['sentimiento'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, stratify=y)

print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10)

X_train_sinopsis_tfidf = vectorizer.fit_transform(X_train['sinopsis_ok'])
X_test_sinopsis_tfidf = vectorizer.transform(X_test['sinopsis_ok'])

print('X_train_sinopsis_tfidf shape: ', X_train_sinopsis_tfidf.shape)
print('X_test_sinopsis_tfidf shape: ', X_test_sinopsis_tfidf.shape)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(penalty='l2')
model.fit(X_train_sinopsis_tfidf, y_train)

y_pred = model.predict(X_test_sinopsis_tfidf)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))

In [ ]:
modeloLR = 'MODELO.pkl'
pickle.dump(model, open(modeloLR,'wb'))

In [ ]:
with open('MODELO.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

In [ ]:
with open('pelis.pk', 'rb') as pelis:
    pelis = pickle.load(pelis)

In [ ]:


pred_sinopsis_tfidf = vectorizer.transform(pelis['sinopsis'])


print('pred_sinopsis_tfidf shape: ', pred_sinopsis_tfidf.shape)


In [ ]:
predicciones = modelo.predict(pred_sinopsis_tfidf)

In [ ]:
pelis['predicciones'] = predicciones

In [ ]:
pelis.head(50)

In [ ]:
pelisok = 'pelisok.pkl'
pickle.dump(pelis, open(pelisok,'wb'))

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

In [62]:
cv= CountVectorizer(stop_words='english', max_features=5000 )

In [63]:
X = cv.fit_transform(df_palabras['datos']).toarray()

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
similarity_matrix = cosine_similarity(X)


In [75]:
def recomendador(movie):
    movie_index = df_palabras[df_palabras['titulo'] == movie].index[0]
    similarity_scores = similarity_matrix[movie_index]
    values = sorted(list(enumerate(similarity_scores)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in values:
        print(df_palabras.iloc[i[0]].titulo)
        
    

In [76]:
df_palabras['titulo'][200]

'The Hunger Games: Mockingjay - Part 1'

In [87]:
recomendador('Avatar')

Beowulf
The Helix... Loaded
Small Soldiers
The Book of Life
Krull
